# EasyOCR 정확도 향상 (계산 방법 수정)

인식한 문자와 라벨링 데이터를 한 글자씩 분리하여 비교하는 방향으로 수정

## 리스트 요소의 문자열을 한 글자씩 분리하여 비교

In [104]:
list1 = ["김해틀"]
tuple1 = ("김해를")

# 리스트와 튜플의 요소를 한 글자씩 분리하여 비교
matching_text = []
for word in list1:
    word_list = list(word)
    for char in word_list:
        if char in tuple1:
            matching_text.append(char)

print(matching_text)

['김', '해']


## set() 함수로 문자열 분리

replace 사용

In [27]:
result_texts = {"김해틀, 아름답게, 시민흘, 행복하게"}
json_test_texts = {"김해를, 아름답게, 시민을, 행복하게"}
result_set = set("".join(result_texts).replace('"', ''))
json_test_set = set("".join(json_test_texts).replace('"', ''))
matching_texts = result_set.intersection(json_test_set)
matching_texts_sorted = sorted(matching_texts, key=lambda x: list(json_test_set).index(x))
print(result_set)
print(json_test_set)
print(matching_texts)
print(matching_texts_sorted)

{'민', '시', '복', '하', '름', '행', '답', '해', ' ', ',', '게', '김', '틀', '아', '흘'}
{'민', '시', '복', '하', '름', '를', '행', '답', '해', ' ', ',', '게', '을', '김', '아'}
{'민', '복', '시', '하', '름', '행', '답', '해', ' ', ',', '게', '김', '아'}
['민', '시', '복', '하', '름', '행', '답', '해', ' ', ',', '게', '김', '아']


## set() 함수를 이용한 비교

In [52]:
result_texts = {'"김해틀', '아름답게', '시민흘', '행복하게"'}
json_test_texts = {'"김해를', '아름답게', '시민을', '행복하게"'}
result_list = []
json_test_list = []
for text_result in sorted(result_texts):
    result_list.extend(list(text_result))
for text_json in sorted(json_test_texts):
    json_test_list.extend(list(text_json))
    
print(result_list)
print(json_test_list)
    
matching_texts = set(result_list).intersection(set(json_test_list))
print(matching_texts)

['"', '김', '해', '틀', '시', '민', '흘', '아', '름', '답', '게', '행', '복', '하', '게', '"']
['"', '김', '해', '를', '시', '민', '을', '아', '름', '답', '게', '행', '복', '하', '게', '"']
{'민', '복', '시', '하', '름', '행', '답', '해', '게', '김', '아', '"'}


## for문을 이용한 비교

중복 허용(?)을 위해서 for문으로 비교

In [55]:
result_texts = {'"김해틀', '아름답게', '시민흘', '행복하게"'}
json_test_texts = {'"김해를', '아름답게', '시민을', '행복하게"'}
result_list = []
json_test_list = []
matching_texts = []

for text_result in sorted(result_texts):
    result_list.extend(list(text_result))
for text_json in sorted(json_test_texts):
    json_test_list.extend(list(text_json))

print(result_list)
print("")
print(json_test_list)
print("")

for i in range(len(result_list)):
    if result_list[i] == json_test_list[i]:
        matching_texts.append(result_list[i])

print(matching_texts)

['"', '김', '해', '틀', '시', '민', '흘', '아', '름', '답', '게', '행', '복', '하', '게', '"']

['"', '김', '해', '를', '시', '민', '을', '아', '름', '답', '게', '행', '복', '하', '게', '"']

['"', '김', '해', '시', '민', '아', '름', '답', '게', '행', '복', '하', '게', '"']


## 테스트 코드 적용 (intersection() 함수 사용)

In [1]:
import easyocr
import json
import os
import time

# 폴더 경로
folder_path_source = '2001_source'
folder_path_label = '2001_label'

# 폴더 안에 있는 모든 파일을 리스트 형태로 가져오기
file_list = os.listdir(folder_path_source)

# 파일 경로와 파일 이름 결합
file_paths = [os.path.join(folder_path_source, file_name) for file_name in file_list]

# 파일 수와 평균 경과 시간, 평균 정확도를 저장할 변수 초기화
file_count = len(file_paths)
total_elapsed_time = 0
total_accuracy = 0


# 파일 개수 지정 (ex) 5개)
file_paths = file_paths[:1]
file_count = 1


# easyocr 모델 로드
reader = easyocr.Reader(['ko','en'], gpu=True)

# 각 파일에 대해 OCR 수행
for file_path in file_paths:
    filename_split = os.path.splitext(os.path.basename(file_path))[0]
    print("파일명 :", os.path.basename(file_path))
    print("인식 실행 중")
    
    # 시작 시간 저장
    start_time = time.time()
    
    # OCR 인식 결과 추출
    result = reader.readtext(folder_path_source+"/"+filename_split+".jpg", width_ths=0.1, adjust_contrast=0.1)
    
    # 인식 경과 시간 계산
    elapsed_time = time.time() - start_time
    
    # 총 경과 시간 계산
    total_elapsed_time += elapsed_time
    
    '''
    # 문자 인식 결과값 출력
    print("<OCR 인식 결과>")
    for i in result:
        print(i[1])
    '''
    
    # 라벨링 데이터(json) 파일 불러오기
    with open(folder_path_label+"/"+filename_split+".json", encoding="UTF8") as file:
        data = json.load(file)
        json_test = data['annotations']
        
        '''
        # 문자 값만 출력
        print("<라벨링 데이터>")
        for k in json_test:
            print(k['annotation.text'])
        '''
    
        # 인식 결과값과 라벨링 데이터 값 추출
        result_texts = {i[1] for i in result}
        json_test_texts = {k['annotation.text'] for k in json_test}
        
        
        result_list = []
        json_test_list = []
        matching_texts = []
        
        for text_result in sorted(result_texts):
            result_list.extend(list(text_result))
        for text_json in sorted(json_test_texts):
            json_test_list.extend(list(text_json))

        # 분리한 인식 문자, 라벨링 문자 / 개수 출력
        print(result_list)
        print(len(result_list))
        print("")
        print(json_test_list)
        print(len(json_test_list))
        print("")
        
        # 분리한 문자 서로 비교
        matching_texts = set(result_list).intersection(set(json_test_list))
        
        # 일치한 문자 / 개수 출력
        print(matching_texts)
        print(len(matching_texts))
        print("")
    
        # 정확도 계산
        percentage = (len(matching_texts) / len(result_list)) * 100
    
        # 일치율 및 정확도, 경과 시간 출력
        if matching_texts:
            
            '''
            print("인식 성공 : ", matching_texts)
            '''
            
            print("정확도 : {:.2f}%".format(percentage))
            print("경과 시간 : {:.2f}초".format(elapsed_time))
            total_accuracy += percentage
        else:
            print("정확한 인식 값이 없습니다.")
            
        print("")

# 총 데이터 수 및 총 소요시간, 평균 경과 시간, 평균 정확도 계산
average_elapsed_time = total_elapsed_time / file_count
average_accuracy = total_accuracy / file_count
total_minutes = int(total_elapsed_time // 60)
total_seconds = int(total_elapsed_time % 60)

# 테스트 결과 출력
print("총 데이터 수 : "+str(file_count)+"개")
print("총 소요 시간 : "+str(total_minutes)+"분 "+str(total_seconds)+"초")
print("평균 인식 시간 : {:.2f}초".format(average_elapsed_time))
print("평균 정확도 : {:.2f}%".format(average_accuracy))

C:\anaconda3\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: [WinError 127] 지정된 프로시저를 찾을 수 없습니다
  warn(f"Failed to load image Python extension: {e}")


파일명 : 5350109-2001-0001-0001.jpg
인식 실행 중
['"', '김', '해', '틀', '#', '@', '이', '3', '쥐', '(', '2', '9', '1', '6', '9', ')', '(', '3', '5', '1', '1', '4', ')', '(', '경', ' ', ' ', ' ', '유', ')', '(', '분', '류', '번', '호', ')', '(', '제', '조', '시', '설', ')', '(', '주', ')', ')', '/', '전', '송', '3', '3', '0', '-', '3', '6', '8', '9', '1', '.', '1', '1', '1', '0', '-', '3', '1', '5', '9', '-', '5', '번', '지', '1', '8', ',', '0', '0', '0', '원', ')', '틀', '2', '.', '2', '0', '0', '1', '.', '2', '0', '0', '만', '원', '이', '하', '의', '2', '개', '월', '이', '내', '에', '3', ',', '2', '0', '4', '.', '0', '3', '.', '4', ' ', '(', ' ', '년', ')', '4', '.', '4', '0', '4', '3', '4', '9', '4', '.', '2', '4', '9', '5', '.', '0', '6', '2', '1', '-', '0', '1', '0', '/', '6', '2', '3', '6', 'y', '7', '5', '4', '.', '1', '2', '8', ',', '0', '0', '0', '원', ',', ':', 'E', '-', 'm', 'a', 'i', 'l', 'I', 'c', 's', '@', 'm', 'a', 'i', 'l', ' ', 'k', 'i', 'm', 'h', 'a', 'e', ' ', 'k', 'y', 'o', 'n', 'g', 'n', 'a', 'm', 'k', 'r'

# 최종 결과

In [103]:
import easyocr
import json
import os
import time

# 폴더 경로
folder_path_source = '2001_source'
folder_path_label = '2001_label'

# 폴더 안에 있는 모든 파일을 리스트 형태로 가져오기
file_list = os.listdir(folder_path_source)

# 파일 경로와 파일 이름 결합
file_paths = [os.path.join(folder_path_source, file_name) for file_name in file_list]

# 파일 수와 평균 경과 시간, 평균 정확도를 저장할 변수 초기화
file_count = len(file_paths)
total_elapsed_time = 0
total_accuracy = 0


# 파일 개수 지정 (ex) 5개)
file_paths = file_paths[:3]
file_count = 3


# easyocr 모델 로드
reader = easyocr.Reader(['ko','en'], gpu=True)

# 각 파일에 대해 OCR 수행
for file_path in file_paths:
    filename_split = os.path.splitext(os.path.basename(file_path))[0]
    print("파일명 :", os.path.basename(file_path))
    print("인식 실행 중")
    
    # 시작 시간 저장
    start_time = time.time()
    
    # OCR 인식 결과 추출
    result = reader.readtext(folder_path_source+"/"+filename_split+".jpg", width_ths=0.1, adjust_contrast=0.1)
    
    # 인식 경과 시간 계산
    elapsed_time = time.time() - start_time
    
    # 총 경과 시간 계산
    total_elapsed_time += elapsed_time
    
    
    # 문자 인식 결과값 출력
    print("")
    print("<OCR 인식 결과>")
    for i in result:
        print(i[1], end=" || ")
    print("")
    print("[인식한 문자] : 총 "+str(len(result))+"개")
    
    
    # 라벨링 데이터(json) 파일 불러오기
    with open(folder_path_label+"/"+filename_split+".json", encoding="UTF8") as file:
        data = json.load(file)
        json_test = data['annotations']
        
        
        # 라벨링 데이터의 문자 값만 출력
        print("")
        print("<라벨링 데이터>")
        for k in json_test:
            print(k['annotation.text'], end=" || ")
        print("")
        print("[라벨링 문자] : 총 "+str(len(json_test))+"개")
        
    
        # 인식 결과값과 라벨링 데이터 값 추출
        result_texts = {i[1] for i in result}
        json_test_texts = {k['annotation.text'] for k in json_test}
        
        result_list = []
        json_test_list = []
        matching_texts = []
        
        # 인식한 문자를 한 글자씩 분리
        for text_result in sorted(result_texts):
            result_list.extend(list(text_result))
        for text_json in sorted(json_test_texts):
            json_test_list.extend(list(text_json))
            
            
        # 분리 결과 출력
        print("")
        print("<인식한 문자 분리 결과>")
        print(result_list)
        print("[분리한 인식 문자] : 총 "+str(len(result_list))+"개")
        print("")
        print("<라벨링 문자 분리 결과>")
        print(json_test_list)
        print("[분리한 라벨링 문자] : 총 "+str(len(json_test_list))+"개")
        
        
        # 한 글자씩 비교하여 일치하는 값을 matching_texts 리스트에 저장
        for item in result_list:
            if item in json_test_list:
                matching_texts.append(item)
                
        # 일치 문자 출력
        print("")
        print("<일치 문자 결과>")
        print(matching_texts)
        print("[일치한 문자] : 총 "+str(len(matching_texts))+"개")
        print("")
    
    
        # 정확도 계산
        percentage = (len(matching_texts) / len(result_list)) * 100
    
        # 정확도, 경과 시간 출력
        if matching_texts:
            print("정확도 : {:.2f}%".format(percentage))
            print("경과 시간 : {:.2f}초".format(elapsed_time))
            total_accuracy += percentage
        else:
            print("정확한 인식 값이 없습니다.")
            
        print("")

# 총 데이터 수 및 총 소요시간, 평균 경과 시간, 평균 정확도 계산
average_elapsed_time = total_elapsed_time / file_count
average_accuracy = total_accuracy / file_count
total_minutes = int(total_elapsed_time // 60)
total_seconds = int(total_elapsed_time % 60)

# 테스트 결과 출력
print("총 데이터 수 : "+str(file_count)+"개")
print("총 소요 시간 : "+str(total_minutes)+"분 "+str(total_seconds)+"초")
print("평균 인식 시간 : {:.2f}초".format(average_elapsed_time))
print("평균 정확도 : {:.2f}%".format(average_accuracy))

파일명 : 5350109-2001-0001-0001.jpg
인식 실행 중

<OCR 인식 결과>
"김해틀 || 아름답게 || 시민흘 || 행복하게" || 김 || 해 || 우 || 621-010/ || 경남 || 김해시 || 부원동 || 623 || 번지/ || 전화(055)330-3363 || /전송330-3689 || 허가민원과 || 과 장 류 보 현 || 공장설립담당주사 || 감 경 석 || 담당자 || 이 창 수 || http:llgimhae.go.kr || E-mail || Ics@mail kimhae kyongnamkr || 문서번호 || 허가55142 || 4043 || 보존기간 || 장 || 시행일자 || 2001. || 4 ( 년) || 공개여부 || (경   유) || 부 시 장 || 발외함 || 수   신 || 부산시 || 사상구 || 감전동 || 159-5번지 || 실 || 장 || 동일중공업주) || 대표 || 담당주사 || 과   장 | || 전   결 || 6y || 참   조 || 기   안 || #@이3쥐 || d심 || 심사자 || 제   목 || 공장설립 || 변경승인 || 통보 || 1. || 귀하께서 || 제출한 || 공장설 립변경승인신청에 || 대하여 || 공업배치 || 및 || 공장설립에 || 관한법률 || 제13조 || 제3항의 || 규정에 || 의거 || 불임과 || 같이 || 승인서률 || 교부하오니, || 조건사항 || 이행에 || 만전올 || 기하시기 || 바갑니다. || 공장설립 || 변경승인 || 내용 || 단위 || mi' || ) || 업   종 || 건축면적 || 부대 || 구분 || 회사명 || 소재지 || 대표자 || 용지면적 || (분류번호) || (제조시설) || 시설 || 당초 || 한림.안하 || 선박구성부분품 || 영광산업사 || 1110-3 || 3,204.0 || 495.0 || 승인 || 외1 필지 || 제조업 || (35114) || 변경 || 동일중공업 || 한림.안하 || 기타물


<OCR 인식 결과>
[제 2 안] || 수   신 || 수신처 || 참조 || 제   목 || 공장설립변경승인사항 || 통보 || 김해시 || 한림면 || 안하리 || 1110-3 || 외1필지상의 || 동일중공업(주) 대표 || 에제 || 공업배치 || 및 || 공장설립에 || 관한법률 || 제13조의 || 규정에 || 의거 || 공장설립 || 변경승인 || 되엇음 || 올 || 통보하니, || 각 || 소관별로 || 담당공무원올 || 지정하여 || 신청지 || 사후관리에 || 만전올 || 기하 || 시기 || 바람니다 . || 0 || 공장설립변경승인 || 내용 : 제 || 1안 || 내용과 || 같음 || 붙임 : || 공장설 립변경승인서 || 1부   끝: || 김  해   시  장 || 수신처 || 세무과 , || 환경보호과 , || 기업육성과 , || 농추산과 , || 한림면장 < || 
[인식한 문자] : 총 52개

<라벨링 데이터>
[제2안] || 수 || 신 || 제 || 목 || 수신처 || 참조 || 공장설립변경승인사항 || 통보 || 김해시 || 한림면 || 안하리 || 1110-3 || 외1필지상의 || 동일중공업(주)대표 || 에게 || 공업배치 || 및 || 공장설립에 || 관한법률 || 제13조의 || 규정에 || 의거 || 공장설립 || 변경승인 || 되었음 || 을 || 통보하니, || 각 || 소관별로 || 담당공무원을 || 지정하여 || 신청지 || 사후관리에 || 만전을 || 기하 || 시기 || 바랍니다. || ο || 공장설립변경승인 || 내용 || : || 제 || 1안 || 내용과 || 같음 || 붙임 || : || 공장설립변경승인서 || 1부 || 끝. || 김 || 해 || 시 || 장 || 수신처 || : || 세무과, || 환경보호과, || 기업육성과, || 농축산과, || 한림면장 || 47 || 
[라벨링 문자] : 총 63개

<인식한 문자 분리 결과>
['0', '1', '1', '1', '0',

# 최종 결과 - 결과만 출력

In [100]:
import easyocr
import json
import os
import time

# 폴더 경로
folder_path_source = '2001_source'
folder_path_label = '2001_label'

# 폴더 안에 있는 모든 파일을 리스트 형태로 가져오기
file_list = os.listdir(folder_path_source)

# 파일 경로와 파일 이름 결합
file_paths = [os.path.join(folder_path_source, file_name) for file_name in file_list]

# 파일 수와 평균 경과 시간, 평균 정확도를 저장할 변수 초기화
file_count = len(file_paths)
total_elapsed_time = 0
total_accuracy = 0


# 파일 개수 지정 (ex) 5개)
file_paths = file_paths[:5]
file_count = 5


# easyocr 모델 로드
reader = easyocr.Reader(['ko','en'], gpu=True)

# 각 파일에 대해 OCR 수행
for file_path in file_paths:
    filename_split = os.path.splitext(os.path.basename(file_path))[0]
    print("파일명 :", os.path.basename(file_path))
    print("인식 실행 중")
    
    # 시작 시간 저장
    start_time = time.time()
    
    # OCR 인식 결과 추출
    result = reader.readtext(folder_path_source+"/"+filename_split+".jpg", width_ths=0.1, adjust_contrast=0.1)
    
    # 인식 경과 시간 계산
    elapsed_time = time.time() - start_time
    
    # 총 경과 시간 계산
    total_elapsed_time += elapsed_time
    
    '''
    # 문자 인식 결과값 출력
    print("")
    print("<OCR 인식 결과>")
    for i in result:
        print(i[1], end=" || ")
    print("")
    print("[인식한 문자] : 총 "+str(len(result))+"개")
    '''
    
    # 라벨링 데이터(json) 파일 불러오기
    with open(folder_path_label+"/"+filename_split+".json", encoding="UTF8") as file:
        data = json.load(file)
        json_test = data['annotations']
        
        '''
        # 라벨링 데이터의 문자 값만 출력
        print("")
        print("<라벨링 데이터>")
        for k in json_test:
            print(k['annotation.text'], end=" || ")
        print("")
        print("[라벨링 문자] : 총 "+str(len(json_test))+"개")
        '''
    
        # 인식 결과값과 라벨링 데이터 값 추출
        result_texts = {i[1] for i in result}
        json_test_texts = {k['annotation.text'] for k in json_test}
        
        result_list = []
        json_test_list = []
        matching_texts = []
        
        # 인식한 문자를 한 글자씩 분리
        for text_result in sorted(result_texts):
            result_list.extend(list(text_result))
        for text_json in sorted(json_test_texts):
            json_test_list.extend(list(text_json))
            
        '''
        # 분리 결과 출력
        print("")
        print("<인식한 문자 분리 결과>")
        print(result_list)
        print("[분리한 인식 문자] : 총 "+str(len(result_list))+"개")
        print("")
        print("<라벨링 문자 분리 결과>")
        print(json_test_list)
        print("[분리한 라벨링 문자] : 총 "+str(len(json_test_list))+"개")
        '''
        
        # 한 글자씩 비교하여 일치하는 값을 matching_texts 리스트에 저장
        for item in result_list:
            if item in json_test_list:
                matching_texts.append(item)
        
        '''
        # 일치 문자 출력
        print("")
        print("<일치 문자 결과>")
        print(matching_texts)
        print("[일치한 문자] : 총 "+str(len(matching_texts))+"개")
        print("")
        '''
    
        # 정확도 계산
        percentage = (len(matching_texts) / len(result_list)) * 100
    
        # 정확도, 경과 시간 출력
        if matching_texts:
            print("정확도 : {:.2f}%".format(percentage))
            print("경과 시간 : {:.2f}초".format(elapsed_time))
            total_accuracy += percentage
        else:
            print("정확한 인식 값이 없습니다.")
            
        print("")

# 총 데이터 수 및 총 소요시간, 평균 경과 시간, 평균 정확도 계산
average_elapsed_time = total_elapsed_time / file_count
average_accuracy = total_accuracy / file_count
total_minutes = int(total_elapsed_time // 60)
total_seconds = int(total_elapsed_time % 60)

# 테스트 결과 출력
print("총 데이터 수 : "+str(file_count)+"개")
print("총 소요 시간 : "+str(total_minutes)+"분 "+str(total_seconds)+"초")
print("평균 인식 시간 : {:.2f}초".format(average_elapsed_time))
print("평균 정확도 : {:.2f}%".format(average_accuracy))

파일명 : 5350109-2001-0001-0001.jpg
인식 실행 중
정확도 : 96.44%
경과 시간 : 2.59초

파일명 : 5350109-2001-0001-0002.jpg
인식 실행 중
정확도 : 84.35%
경과 시간 : 1.05초

파일명 : 5350109-2001-0001-0004.jpg
인식 실행 중
정확도 : 96.22%
경과 시간 : 1.93초

파일명 : 5350109-2001-0001-0005.jpg
인식 실행 중
정확도 : 95.25%
경과 시간 : 2.09초

파일명 : 5350109-2001-0001-0006.jpg
인식 실행 중
정확도 : 81.13%
경과 시간 : 0.92초

총 데이터 수 : 5개
총 소요 시간 : 0분 8초
평균 인식 시간 : 1.72초
평균 정확도 : 90.68%


# 최종 결과 - 샘플 데이터 전체 테스트

In [2]:
import easyocr
import json
import os
import time

# 폴더 경로
folder_path_source = '2001_source'
folder_path_label = '2001_label'

# 폴더 안에 있는 모든 파일을 리스트 형태로 가져오기
file_list = os.listdir(folder_path_source)

# 파일 경로와 파일 이름 결합
file_paths = [os.path.join(folder_path_source, file_name) for file_name in file_list]

# 파일 수와 평균 경과 시간, 평균 정확도를 저장할 변수 초기화
file_count = len(file_paths)
total_elapsed_time = 0
total_accuracy = 0

'''
# 파일 개수 지정 (ex) 5개)
file_paths = file_paths[:5]
file_count = 5
'''

# easyocr 모델 로드
reader = easyocr.Reader(['ko','en'], gpu=True)

# 각 파일에 대해 OCR 수행
for file_path in file_paths:
    filename_split = os.path.splitext(os.path.basename(file_path))[0]
    print("파일명 :", os.path.basename(file_path))
    print("인식 실행 중")
    
    # 시작 시간 저장
    start_time = time.time()
    
    # OCR 인식 결과 추출
    result = reader.readtext(folder_path_source+"/"+filename_split+".jpg", width_ths=0.1, adjust_contrast=0.1)
    
    # 인식 경과 시간 계산
    elapsed_time = time.time() - start_time
    
    # 총 경과 시간 계산
    total_elapsed_time += elapsed_time
    
    '''
    # 문자 인식 결과값 출력
    print("")
    print("<OCR 인식 결과>")
    for i in result:
        print(i[1], end=" || ")
    print("")
    print("[인식한 문자] : 총 "+str(len(result))+"개")
    '''
    
    # 라벨링 데이터(json) 파일 불러오기
    with open(folder_path_label+"/"+filename_split+".json", encoding="UTF8") as file:
        data = json.load(file)
        json_test = data['annotations']
        
        '''
        # 라벨링 데이터의 문자 값만 출력
        print("")
        print("<라벨링 데이터>")
        for k in json_test:
            print(k['annotation.text'], end=" || ")
        print("")
        print("[라벨링 문자] : 총 "+str(len(json_test))+"개")
        '''
    
        # 인식 결과값과 라벨링 데이터 값 추출
        result_texts = {i[1] for i in result}
        json_test_texts = {k['annotation.text'] for k in json_test}
        
        result_list = []
        json_test_list = []
        matching_texts = []
        
        # 인식한 문자를 한 글자씩 분리
        for text_result in sorted(result_texts):
            result_list.extend(list(text_result))
        for text_json in sorted(json_test_texts):
            json_test_list.extend(list(text_json))
            
        '''
        # 분리 결과 출력
        print("")
        print("<인식한 문자 분리 결과>")
        print(result_list)
        print("[분리한 인식 문자] : 총 "+str(len(result_list))+"개")
        print("")
        print("<라벨링 문자 분리 결과>")
        print(json_test_list)
        print("[분리한 라벨링 문자] : 총 "+str(len(json_test_list))+"개")
        '''
        
        # 한 글자씩 비교하여 일치하는 값을 matching_texts 리스트에 저장
        for item in result_list:
            if item in json_test_list:
                matching_texts.append(item)
        
        '''
        # 일치 문자 출력
        print("")
        print("<일치 문자 결과>")
        print(matching_texts)
        print("[일치한 문자] : 총 "+str(len(matching_texts))+"개")
        print("")
        '''
    
        # 정확도 계산
        percentage = (len(matching_texts) / len(result_list)) * 100
    
        # 정확도, 경과 시간 출력
        if matching_texts:
            print("정확도 : {:.2f}%".format(percentage))
            print("경과 시간 : {:.2f}초".format(elapsed_time))
            total_accuracy += percentage
        else:
            print("정확한 인식 값이 없습니다.")
            
        print("")

# 총 데이터 수 및 총 소요시간, 평균 경과 시간, 평균 정확도 계산
average_elapsed_time = total_elapsed_time / file_count
average_accuracy = total_accuracy / file_count
total_minutes = int(total_elapsed_time // 60)
total_seconds = int(total_elapsed_time % 60)

# 테스트 결과 출력
print("총 데이터 수 : "+str(file_count)+"개")
print("총 소요 시간 : "+str(total_minutes)+"분 "+str(total_seconds)+"초")
print("평균 인식 시간 : {:.2f}초".format(average_elapsed_time))
print("평균 정확도 : {:.2f}%".format(average_accuracy))

파일명 : 5350109-2001-0001-0001.jpg
인식 실행 중
정확도 : 96.44%
경과 시간 : 2.96초

파일명 : 5350109-2001-0001-0002.jpg
인식 실행 중
정확도 : 84.35%
경과 시간 : 1.13초

파일명 : 5350109-2001-0001-0004.jpg
인식 실행 중
정확도 : 96.22%
경과 시간 : 2.12초

파일명 : 5350109-2001-0001-0005.jpg
인식 실행 중
정확도 : 95.25%
경과 시간 : 2.19초

파일명 : 5350109-2001-0001-0006.jpg
인식 실행 중
정확도 : 81.13%
경과 시간 : 0.97초

파일명 : 5350109-2001-0001-0009.jpg
인식 실행 중
정확도 : 96.73%
경과 시간 : 2.06초

파일명 : 5350109-2001-0001-0010.jpg
인식 실행 중
정확도 : 97.00%
경과 시간 : 1.89초

파일명 : 5350109-2001-0001-0015.jpg
인식 실행 중
정확도 : 95.68%
경과 시간 : 2.11초

파일명 : 5350109-2001-0001-0016.jpg
인식 실행 중
정확도 : 72.73%
경과 시간 : 0.70초

파일명 : 5350109-2001-0001-0017.jpg
인식 실행 중
정확도 : 94.14%
경과 시간 : 2.40초

파일명 : 5350109-2001-0001-0018.jpg
인식 실행 중
정확도 : 96.81%
경과 시간 : 2.18초

파일명 : 5350109-2001-0001-0019.jpg
인식 실행 중
정확도 : 73.86%
경과 시간 : 0.73초

파일명 : 5350109-2001-0001-0020.jpg
인식 실행 중
정확도 : 95.91%
경과 시간 : 2.99초

파일명 : 5350109-2001-0001-0021.jpg
인식 실행 중
정확도 : 95.32%
경과 시간 : 1.68초

파일명 : 5350109-2001-0001-0022.jpg
인

정확도 : 81.63%
경과 시간 : 1.04초

파일명 : 5350109-2001-0001-0466.jpg
인식 실행 중
정확도 : 96.57%
경과 시간 : 2.30초

파일명 : 5350109-2001-0001-0467.jpg
인식 실행 중
정확도 : 94.16%
경과 시간 : 2.45초

파일명 : 5350109-2001-0001-0468.jpg
인식 실행 중
정확도 : 96.65%
경과 시간 : 0.98초

파일명 : 5350109-2001-0001-0469.jpg
인식 실행 중
정확도 : 95.06%
경과 시간 : 2.16초

파일명 : 5350109-2001-0001-0470.jpg
인식 실행 중
정확도 : 95.89%
경과 시간 : 2.20초

파일명 : 5350109-2001-0001-0471.jpg
인식 실행 중
정확도 : 73.56%
경과 시간 : 0.69초

파일명 : 5350109-2001-0001-0472.jpg
인식 실행 중
정확도 : 94.88%
경과 시간 : 2.36초

파일명 : 5350109-2001-0001-0473.jpg
인식 실행 중
정확도 : 96.60%
경과 시간 : 1.17초

파일명 : 5350109-2001-0001-0479.jpg
인식 실행 중
정확도 : 94.32%
경과 시간 : 2.51초

파일명 : 5350109-2001-0001-0480.jpg
인식 실행 중
정확도 : 80.91%
경과 시간 : 1.01초

파일명 : 5350109-2001-0001-0481.jpg
인식 실행 중
정확도 : 96.35%
경과 시간 : 2.61초

파일명 : 5350109-2001-0001-0482.jpg
인식 실행 중
정확도 : 96.53%
경과 시간 : 2.49초

파일명 : 5350109-2001-0001-0483.jpg
인식 실행 중
정확도 : 73.86%
경과 시간 : 0.74초

파일명 : 5350109-2001-0001-0484.jpg
인식 실행 중
정확도 : 97.10%
경과 시간 : 2.10초

파일명 : 

### 개선 결과

정확도는 기존의 계산 방법으로 나온 60% 대에서 90% 대까지 약 30% 향상됨.

문제는 이 또한 단순히 리스트 값으로 단순 비교한 것이기에 실제 이미지에서 같은 위치의 문자와 인식한 결과인 것인지, 중복된 값으로 인식한 것인지는 확인해봐야 할 듯 싶다.
또한 정확도 계산 방법만 수정한 것이기에 실질적인 문자 인식률이 향상된 것은 아니다.